# Проект "Компьютер говорит нет"

# Импорт библиотек

In [1]:
from pandas import Series
import pandas as pd
import numpy as np
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import recall_score, precision_score, f1_score, log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve, average_precision_score
from datetime import date
from datetime import datetime, timedelta

warnings.filterwarnings("ignore")

client_id - идентификатор клиента

education - уровень образования

sex - пол заемщика

age - возраст заемщика

car - флаг наличия автомобиля

car_type - флаг автомобиля иномарки

decline_app_cnt - количество отказанных прошлых заявок

good_work - флаг наличия “хорошей” работы

bki_request_cnt - количество запросов в БКИ

home_address - категоризатор домашнего адреса

work_address - категоризатор рабочего адреса

income - доход заемщика

foreign_passport - наличие загранпаспорта

sna - связь заемщика с клиентами банка

first_time - давность наличия информации о заемщике

score_bki - скоринговый балл по данным из БКИ

region_rating - рейтинг региона

app_date - дата подачи заявки

default - флаг дефолта по кредиту

In [2]:
PATH_to_file = '/kaggle/input/sf-dst-scoring/'
train = pd.read_csv(PATH_to_file+'train.csv')
test = pd.read_csv(PATH_to_file+'test.csv')

In [3]:
# Посмотрим на слотбцы датафрейма. 
train.dtypes

client_id             int64
app_date             object
education            object
sex                  object
age                   int64
car                  object
car_type             object
decline_app_cnt       int64
good_work             int64
score_bki           float64
bki_request_cnt       int64
region_rating         int64
home_address          int64
work_address          int64
income                int64
sna                   int64
first_time            int64
foreign_passport     object
default               int64
dtype: object

In [4]:
# Количество пропушенных значений в разрезе признаков по обоим датасетам

train.isna().sum()

client_id             0
app_date              0
education           307
sex                   0
age                   0
car                   0
car_type              0
decline_app_cnt       0
good_work             0
score_bki             0
bki_request_cnt       0
region_rating         0
home_address          0
work_address          0
income                0
sna                   0
first_time            0
foreign_passport      0
default               0
dtype: int64

In [5]:
test.isna().sum()

client_id             0
app_date              0
education           171
sex                   0
age                   0
car                   0
car_type              0
decline_app_cnt       0
good_work             0
score_bki             0
bki_request_cnt       0
region_rating         0
home_address          0
work_address          0
income                0
sna                   0
first_time            0
foreign_passport      0
dtype: int64

In [6]:
Так как единственных столбец который содержит пропуски - education, попробуем обработать его.

SyntaxError: invalid syntax (<ipython-input-6-2826dcf76a95>, line 1)

In [ ]:
train['education'].hist()

In [ ]:
test['education'].hist()

In [ ]:
# Значение SCH в обоих датасетах является наиболее часто встречаемым поэтому сделаем заполнение для train и test 

train_mode = train['education'].mode()
train['education'].fillna(train_mode[0], inplace=True)
train['education'].value_counts()

In [ ]:
test_mode = test['education'].mode()
test['education'].fillna(test_mode[0], inplace=True)
test['education'].value_counts()

Проанализируем каждый столбец на распределение и наличие выбросов

In [ ]:
# Столбец с идентификатором клиента

train['client_id'].value_counts()
train['client_id'].sort_values()
#train['client_id'].hist()

In [ ]:
test['client_id'].value_counts()
test['client_id'].sort_values()

Все идентификаторы уникальны, т.е. нет повторяющихся и отрицательых значений

In [ ]:
# Пол клиента

sns.countplot(x = train['sex'], data = train)
train['sex'].value_counts()

In [ ]:
sns.countplot(x = test['sex'], data = test)
test['sex'].value_counts()

Среди клиентов выбросов и некорректных знаений не содержит. Женщин среди клиентов немного больше чем мужчин

In [ ]:
# Возраст клиента

#sns.countplot(x = train['age'], data = train)
train['age'].hist()
train['age'].groupby(train['age']).count()
train['age'].value_counts()> 2000

По возрасту клиентов, видно, что самые молодые клиенты в возрасте 21 год, самый старый - 72 года (таких 2), средний возраст заемщика - 39 лет. Основная часть заемщиков находится в диапазоне от 25 до 39 лет 


In [ ]:
# Наличие автомобиля

sns.countplot(x = train['car'], data = train)
train['car'].value_counts()

In [ ]:
sns.countplot(x = test['car'], data = test)
test['car'].value_counts()

Столбец наличие автомобиля, также выбросов не имеет все значения заполнены, клиентов не имеющих автомобиль в 2 раза больше, чем клиентов имеющих автомобиль

In [ ]:
# Автомобиль иномарка

sns.countplot(x = train['car_type'], data = train)
train['car_type'].value_counts()

In [ ]:
sns.countplot(x = test['car_type'], data = test)
test['car_type'].value_counts()

In [ ]:
train[(train['car'] == 'N') & (train['car_type'] == 'Y')]

In [ ]:
test[(test['car'] == 'N') & (test['car_type'] == 'Y')]

Столбец на наличие иномарки заполнен без пропусков, видно, что менее 20% заемщиков имеют в наличии иномарки. Данный автомобиль связан с предыдущим столбцом, наличием автомобиля. Если заемщик не имеет автомобиля, то соответственно столбец наличие иномарки должен быть заполнен также со значением "N", иначе будет считаться как не корректное заполнение столбца.  

Можно сразу создать новый признак по наличию автомобиля, и наличию иномарки. Будем выставлять 0 - у кого отсутствует автомобиль, 1 - автомобиль отечественного производства, 2 - автомобиль иномарка

In [ ]:
c = []

for i in train.car.index:
    if train.car[i] == 'N':
        c.append(0)
    elif train.car_type[i] == 'N':
        c.append(1)
    else:
        c.append(2)

In [ ]:
train['car_all'] = pd.Series(c)
train.car_all.value_counts()

In [ ]:
c = []

for i in test.car.index:
    if test.car[i] == 'N':
        c.append(0)
    elif test.car_type[i] == 'N':
        c.append(1)
    else:
        c.append(2)

In [ ]:
test['car_all'] = pd.Series(c)
test.car_all.value_counts()

In [ ]:
# Количество отказов 

sns.countplot(x = train['decline_app_cnt'], data = train)
train['decline_app_cnt'].value_counts()

In [ ]:
sns.countplot(x = test['decline_app_cnt'], data = test)
test['decline_app_cnt'].value_counts()

Количество отказов показывает, что большая часть клиентов не имеет отказов, от 15 до 33 отказов имеют по 1 клиенту.

In [ ]:
# Наличие хорошей работы

sns.countplot(x = train['good_work'], data = train)
train['good_work'].value_counts()

In [ ]:
sns.countplot(x = test['good_work'], data = test)
test['good_work'].value_counts()

Столбец на наличие хорошей работы имеет значений 0 и 1. Не понятно, что считается хорошей работой, как она определяется. Видно, что большая часть клиентов, не имеют эту хорошую работу. Выбросов в данном столбце, нет, пропущенных значений нет.

In [ ]:
# количество запросов в БКИ

sns.countplot(x = train['bki_request_cnt'], data = train)
train['bki_request_cnt'].value_counts()

In [ ]:
sns.countplot(x = test['bki_request_cnt'], data = test)
test['bki_request_cnt'].value_counts()

Большинство клиентов либо вообще не имеют запросов в БКИ

In [ ]:
# Категоризатор домашнего адреса

sns.countplot(x = train['home_address'], data = train)
train['home_address'].value_counts()

In [ ]:
sns.countplot(x = test['home_address'], data = test)
test['home_address'].value_counts()

In [ ]:
# Категоризатор рабочего адреса

sns.countplot(x = train['work_address'], data = train)
train['work_address'].value_counts()

In [ ]:
sns.countplot(x = train['work_address'], data = test)
test['work_address'].value_counts()

Категоризатор домашнего и рабочего адреса имеет категориальный признак и принимают значения от 1 до 3. Возможно он означает город, пригород и иной какой адрес проживания, и работы. 

In [ ]:
# Доход клиента

print(train['income'].mean())
print(train['income'].min())
print(train['income'].max())

In [ ]:
# Сразу можно добавить новый признак, который будет показывать, уровень зарплаты выше среднего или ниже. Будем проставлять соответственно 1 или 0
train['income_mean'] = train.income.apply(lambda x: 1 if x > train.income.mean() else 0)

In [ ]:
# Выполним тоже самое для тестовой выборки
test['income_mean'] = test.income.apply(lambda x: 1 if x > test.income.mean() else 0)

In [ ]:
print(test['income'].mean())
print(test['income'].min())
print(test['income'].max())

По доходу клиента, видно, что средний доход равен - 40000руб., минимальный - 1000, максимальный - 1млн.

In [ ]:
# Связь заемщика с сотрудником банка

sns.countplot(x = train['sna'], data = train)
train['sna'].value_counts()

In [ ]:
sns.countplot(x = test['sna'], data = test)
test['sna'].value_counts()

Связь заемщика с сотрудником банка указана в значениях от 1 до 4. У больше половины клиентов указано значение 1, возможно это вообще отсутствие связи с сотрудников, либо связь с сотрудников банка минимальна.

In [ ]:
# давность наличия информации о заемщике

sns.countplot(x = train['first_time'], data = train)
train['first_time'].value_counts()

In [ ]:
sns.countplot(x = test['first_time'], data = test)
test['first_time'].value_counts()

Давность наличия информации о заемщике имеет также категориальный признак со значениями от 1 до 4. Почти половина клиентов имееет значений равно = 3.

In [ ]:
# скоринговый балл по данным из БКИ

train['score_bki'].hist()
print('Максимальный скорринговый бал равен =', train['score_bki'].max(), 'Минимальный скоринговый бал равен = ', train['score_bki'].min())

In [ ]:
test['score_bki'].hist()
print('Максимальный скорринговый бал равен =', test['score_bki'].max(), 'Минимальный скоринговый бал равен = ', test['score_bki'].min())

Большинство клиентов имеют скориновый балл от -2.5 до -1.5 Не знаем как он рассчитавыется от чего зависит и какие значения являются нормальными, поэтому оставим пока как есть. 

In [ ]:
# рейтинг региона

sns.countplot(x = train['region_rating'], data = train)
train['region_rating'].value_counts()

In [ ]:
sns.countplot(x = test['region_rating'], data = test)
test['region_rating'].value_counts()

Столбец рейтинга имеет категориальный признак. Имеет значения от 20 до 80. Больше половины клиентов находятся в рейтинге региона имеюзего значение 50 и 60

In [ ]:
# налииче загранпаспорта

sns.countplot(x = train['foreign_passport'], data = train)
train['foreign_passport'].value_counts()

In [ ]:
sns.countplot(x = test['foreign_passport'], data = test)
test['foreign_passport'].value_counts()

Столбец наличие заграпаспорта имеет бинарный признак. Видно что большинство клиентов указывают об отсутствии загранпаспорта.

Для обучения нашего датасета, необходимо перевести все признаки в числовой вид. У нас есть бинарные признаки ('sex', 'car', 'car_type', 'foreign_passport', 'good_work') а также признак в формате даты.

In [ ]:
# Выполним преобразование даты с мопощью модуля datetime

train['app_date'] = train.app_date.apply(lambda x: datetime.strptime(x, '%d%b%Y'))
test['app_date'] = test.app_date.apply(lambda x: datetime.strptime(x, '%d%b%Y'))

In [ ]:
# Создадим 3 столбца, куда поместим день, месяц а также день недели. (Так как год указан 2014, его брать не будем)

train['day'] = train.app_date.apply(lambda s: s.day)
train['month'] = train.app_date.apply(lambda s: s.month)
train['weekday'] = train.app_date.apply(lambda s: s.weekday())

test['day'] = test.app_date.apply(lambda s: s.day)
test['month'] = test.app_date.apply(lambda s: s.month)
test['weekday'] = test.app_date.apply(lambda s: s.weekday())

In [ ]:
# Выведем разницу дней относительно начальной датой подачи заявки
date_min = train['app_date'].min()
train['date_count'] = train.app_date.apply(lambda x: (x - date_min).days)

In [ ]:
date_min = test['app_date'].min()
test['date_count'] = test.app_date.apply(lambda x: (x - date_min).days)

In [ ]:
# Посмотрим на количество уникальных значений по признакам

train.nunique()

In [ ]:
test.nunique()

Сделаем разбиение на бинарные, категориальные и количественные группы все признаки

In [ ]:
bin_cols = ['sex', 'foreign_passport', 'good_work', 'income_mean']
cat_cools = ['education', 'car', 'car_type', 'car_all', 'home_address', 'work_address', 'region_rating', 'sna', 'first_time', 'weekday', 'month']
num_cols = ['age', 'decline_app_cnt', 'income', 'bki_request_cnt', 'score_bki', 'day', 'date_count']

Представим бинарные признаки в числовом формате

In [ ]:
label_encoder = LabelEncoder()

for column in bin_cols:
    train[column] = label_encoder.fit_transform(train[column])

In [ ]:
label_encoder = LabelEncoder()

for column in bin_cols:
    test[column] = label_encoder.fit_transform(test[column])

In [ ]:
for col in num_cols:
    plt.figure()
    sns.distplot(train[col], kde=False, rug=False)
    plt.title(col)
    plt.show()

Проверим количественные признаки на выбросы

In [ ]:
for col in num_cols:
    IQR = train[col].quantile(0.75) - train[col].quantile(0.25)
    perc25 = train[col].quantile(0.25)  # 25-й перцентиль
    perc75 = train[col].quantile(0.75)  # 75-й перцентиль

    print(
        f'Название столбца:{col} '
        '25-й перцентиль: {},'.format(perc25),
        '75-й перцентиль: {},'.format(perc75),
        "IQR: {}, ".format(IQR),
        "Границы выбросов: [{f}, {l}].".format(f=perc25 - 1.5*IQR,
                                               l=perc75 + 1.5*IQR))

для тестовой выборки

In [ ]:
for col in num_cols:
    IQR = test[col].quantile(0.75) - test[col].quantile(0.25)
    perc25 = test[col].quantile(0.25)  # 25-й перцентиль
    perc75 = test[col].quantile(0.75)  # 75-й перцентиль

    print(
        f'Название столбца:{col} '
        '25-й перцентиль: {},'.format(perc25),
        '75-й перцентиль: {},'.format(perc75),
        "IQR: {}, ".format(IQR),
        "Границы выбросов: [{f}, {l}].".format(f=perc25 - 1.5*IQR,
                                               l=perc75 + 1.5*IQR))

In [ ]:
for i in num_cols:
    plt.figure()
    sns.distplot(test[i][test[i] > 0].dropna(), kde = False, rug=False)
    plt.title(i)
    plt.show()

In [ ]:
num_cols_log = ['age', 'decline_app_cnt',
                'bki_request_cnt', 'income', 'app_age']
for col in num_cols_log:
    df[col] = np.log(df[col] + 1)

Категориальные признаки переведем в числовой формат, для дальнейшей обработки

In [ ]:
for column in cat_cools:
    train[column] = label_encoder.fit_transform(train[column])

In [ ]:
for column in cat_cools:
    test[column] = label_encoder.fit_transform(test[column])

Построим тепловыю карту корреляции признаков

In [ ]:
sns.heatmap(train[num_cols].corr().abs(), vmin=0, vmax=1, annot=True)

Тепловая карта для числовых признаков высокой корреляции непоказывает

In [ ]:
sns.heatmap(train[cat_cools].corr().abs(), vmin=0, vmax=1, annot=True)

Среди категориальных признаков высокая корреляция наблюдается между признаками домашнего и рабочего адреса. Можно удалить один из признаков адреса в зависимости от значимости

Постоим график, который покажет значимость бинарных и категориальных признаков

In [ ]:
imp_cat = Series(mutual_info_classif(train[bin_cols + cat_cools], train['default'],
                                     discrete_features =True), index = bin_cols + cat_cools)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

Видно, что большую значимость среди признаков показывает связь заемщика с клиентом банка, также давность наличия информации о заемщике

In [ ]:
imp_num = pd.Series(f_classif(train[num_cols], train['default'])[0], index=num_cols)
imp_num.sort_values(inplace=True)
imp_num.plot(kind='barh')
plt.title('Значимость числовых признаков')

Среди числовых признаком наиболее значимым признаком является скоринговый балл. 

In [ ]:
X_train_cat = OneHotEncoder(sparse = False).fit_transform(train[cat_cools].values)
Y_test_cat = OneHotEncoder(sparse = False).fit_transform(test[cat_cools].values)

Стандартицируем числовые признаки в обоих выборках

In [ ]:
X_num = StandardScaler().fit_transform(train[num_cols].values)
Y_num = StandardScaler().fit_transform(test[num_cols].values)

Объединим

In [ ]:
X = np.hstack([X_num, train[bin_cols].values, X_train_cat])
Y = train['default'].values

#id_test = test['client_id']
#test = np.hstack([Y_num, test[bin_cols].values, X_test_cat])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

Подберем гиперпараметры

In [ ]:
# Добавим типы регуляризации
penalty = ['l1', 'l2']

# Зададим ограничения для параметра регуляризации
C = np.logspace(0, 4, 10)

# Создадим гиперпараметры
hyperparameters = dict(C=C, penalty=penalty)

model = LogisticRegression()
model.fit(X_train, y_train)

# Создаем сетку поиска с использованием 5-кратной перекрестной проверки
clf = GridSearchCV(model, hyperparameters, cv=5, verbose=0)

best_model = clf.fit(X_train, y_train)

# View best hyperparameters
print('Лучшее Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Лучшее C:', best_model.best_estimator_.get_params()['C'])

Обучим модель с полученными гиперпараметрами

In [ ]:
model = LogisticRegression(penalty='l2', C=1.0, max_iter=1000)
model.fit(X_train, y_train)

In [ ]:
probs = model.predict_proba(X_test)
probs = probs[:,1]


fpr, tpr, threshold = roc_curve(y_test, probs)
roc_auc = roc_auc_score(y_test, probs)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
Y_predicted = model.predict(X_test)

In [ ]:
print('accuracy_score:',accuracy_score(y_test, Y_predicted))
print('precision_score:',precision_score(y_test, Y_predicted))
print('recall_score:',recall_score(y_test, Y_predicted))
print('f1_score:',f1_score(y_test, Y_predicted))

Построим матрицу ошибок

In [ ]:
from sklearn.metrics import confusion_matrix
print (confusion_matrix(y_test, Y_predicted))
sns.heatmap(confusion_matrix(y_test, Y_predicted), annot=True, annot_kws={"size": 20}, fmt='', cmap= 'Pastel1', cbar = False, \
                 xticklabels = ['Дефолт','Не дефолт'], yticklabels= ['Дефолт','Не дефолт'])
plt.title('Матрица ошибок для default')
plt.show()

Выкладываем результат на Kaggle

In [ ]:
id_test = test['client_id']
test = np.hstack([Y_num, test[bin_cols].values, Y_test_cat])

Подберем гиперпараметры

In [ ]:
model = LogisticRegression(penalty = 'l2', C=1.0, max_iter=1000)
model.fit(X, Y)
probs = model.predict_proba(test)
probs = probs[:,1]

In [ ]:
my_submission = pd.DataFrame({'client_id': id_test,
                              'default': probs})
my_submission.to_csv('submission.csv', index=False)

my_submission.head(10)